# Part 2: Simple model

## Task 0: Grouping labels

In [1]:
import pandas as pd

src = 'data/995,000_rows.csv'
# src = 'data/news_sample_cleaned.csv'
df = pd.read_csv(src)

/var/folders/r7/pckx8d3j3_7fnbs31bc_1_r80000gn/T/ipykernel_98750/2174348663.py:5: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(src)


In [2]:
type_dist = df['type'].value_counts(normalize=True) * 100
print(type_dist)

type
reliable                      23.074406
political                     20.535803
bias                          14.065670
fake                          11.072788
conspiracy                    10.273708
rumor                          5.959055
unknown                        4.596005
unreliable                     3.730097
clickbait                      2.893961
junksci                        1.482242
satire                         1.389338
hate                           0.926823
2018-02-10 13:43:39.521661     0.000106
Name: proportion, dtype: float64


In [3]:
# domain_dist = df['domain'].value_counts(normalize=True) * 100
# print(domain_dist)

In [4]:
# remove types
omitted_types = ['unknown',
                 'bias',
                 'conspiracy',
                 'political',
                 'satire',
                 'rumor',
                 '2018-02-10 13:43:39.521661'
                 ]

for i in range(len(omitted_types)):
    df = df.drop(df[df['type'] == omitted_types[i]].index)

# query a type
# df.query(' type == "fake" ')


In [5]:
fake_types = {'fake', 
              'conspiracy', 
              'unreliable', 
              'junksci',
              'clickbait',
              'hate' 
}

reliable_types = {'reliable'}

# assign new label to 
def change_to_fake(type: str) -> str:
    if type in fake_types:
        return 'fake'
    else:
        return type
    
df['type'] = df['type'].apply(change_to_fake)

In [6]:
# destribution of fake vs. reliable after re-labelling
type_dist = df['type'].value_counts(normalize=True) * 100
print(type_dist)

type
reliable    53.437324
fake        46.562676
Name: proportion, dtype: float64
